In [135]:
from nltk.corpus import wordnet as wn
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
import random
import sys

In [136]:
#get list of all synset wnids
wnids_url='http://www.image-net.org/api/text/imagenet.synset.obtain_synset_list'
wnids = requests.get(wnids_url).content.decode('utf-8')
wnids=wnids.split('\n')
print("Number of synsets: ",len(wnids))

Number of synsets:  21844


In [162]:
#is-a relation hierarchy 
#parent-child wnids
url='http://www.image-net.org/archive/wordnet.is_a.txt'
wnids_pc = requests.get(url).content.decode('utf-8')
wnids_pc=wnids_pc.split('\n')
print('Number of parent-child relations: ',len(wnids_pc))

Number of parent-child relations:  75851


In [138]:
user_agent_url = 'http://www.image-net.org/api/xml/structure_released.xml'
xml_data = requests.get(user_agent_url).content

data=[]
class XML2DataFrame:
    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        """Return a list of dictionaries from the text
         and attributes of the children under this XML root."""
        global data
        
        for child in iter(root):
            self.parse_element(child)            
        return data

    def parse_element(self, element,parsed=None):
        """ Collect {key:attribute} and {tag:text} from thie XML
         element and all its children into a single dictionary of strings."""
        global data
        if parsed is None:
            parsed = dict()

        for key in element.keys():
            parsed[key] = element.attrib.get(key)

        if element.text:
            parsed[element.tag] = element.text
  
        #print(parsed)
        data.append(parsed)
        """ Apply recursion"""
        
        for child in list(element):
            self.parse_element(child)
           
        
        #print(parsed)
        return parsed

    def process_data(self):
        """ Initiate the root XML, parse it, and return a dataframe"""
        structure_data = self.parse_root(self.root)
        print(len(structure_data))
        return pd.DataFrame(structure_data)


In [139]:
xml2df = XML2DataFrame(xml_data)
xml_dataframe = xml2df.process_data()
xml_dataframe=xml_dataframe.drop(labels=['releaseData','synset'],axis=1)


60943


In [140]:
xml_dataframe=xml_dataframe.iloc[2:,:]
xml_dataframe.head(5)

,gloss,wnid,words
2,(botany) a living organism lacking the power of locomotion,n00017222,"plant, flora, plant life"
3,photosynthetic or plant constituent of plankton; mainly unicellular algae,n01383896,phytoplankton
4,unicellular algae,n01384084,planktonic algae
5,microscopic unicellular marine or freshwater colonial alga having cell walls impregnated with silica,n01401106,diatom
6,microscopic plants; bacteria are often considered to be microflora,n11530008,microflora


In [141]:
xml_dataframe.reset_index(drop=True,inplace=True)
xml_dataframe=xml_dataframe.drop_duplicates(keep='first')
xml_dataframe.head(),xml_dataframe.shape

(                                                                                                  gloss  \
 0                                            (botany) a living organism lacking the power of locomotion   
 1                             photosynthetic or plant constituent of plankton; mainly unicellular algae   
 2                                                                                     unicellular algae   
 3  microscopic unicellular marine or freshwater colonial alga having cell walls impregnated with silica   
 4                                    microscopic plants; bacteria are often considered to be microflora   
 
         wnid                     words  
 0  n00017222  plant, flora, plant life  
 1  n01383896             phytoplankton  
 2  n01384084          planktonic algae  
 3  n01401106                    diatom  
 4  n11530008                microflora  , (32296, 3))

In [189]:

synsets=xml_dataframe['words'].tolist()

rose_words=[]
for synset in synsets:
    if('rose' in synset):
        rose_words.append(synset)
        
#pick a random rose synset.
wnid=random.randint(0,len(rose_words))
rose_synset=rose_words[wnid]
print(rose_synset)

rose apple


In [190]:
#get image names, image urls given the wnid from imagenet.
#first get its wnid and gloss.
rose_=xml_dataframe[xml_dataframe['words']==rose_synset]
rose_['gloss'].to_csv(sys.stdout)
rose_['wnid'].to_csv(sys.stdout)
rose_['words'].to_csv(sys.stdout)
print()

wnid=rose_['wnid'].tolist()[0]
gloss=rose_['gloss'].tolist()[0]
words=rose_['words'].tolist()[0]

5397,fragrant oval yellowish tropical fruit used in jellies and confections
5397,n07770034
5397,rose apple



In [185]:
image_names_urls='http://www.image-net.org/api/text/imagenet.synset.geturls.getmapping?wnid=%s'%wnid
names_urls= requests.get(image_names_urls).content.decode('utf-8')
names_urls=names_urls.split('\r\n')
def map_names_urls(x):
    x=x.split()
    #print(x)
    return {'name':x[0],'url':x[1]}

names_urls=[map_names_urls(x) for x in names_urls if x!='']
names_urls=pd.DataFrame(names_urls)
names_urls.iloc[0,:].to_csv(sys.stdout)

name,n02175569_18
url,http://farm4.static.flickr.com/3268/2795079649_88b87eb612.jpg?v=0


In [156]:
#write xml_data to csv file.
xml_dataframe.to_csv('./words_wnids_gloss_imagenet.csv')
x=pd.read_csv('./words_wnids_gloss_imagenet.csv')
print(x.shape)
x=x.dropna(how='any')
print(x.shape)

(32296, 4)
(32295, 4)


In [163]:
#write wnids_pc to csv file.
def map_parent_child(x):
    x=x.split()
    return {'parent':x[0],'child':x[1]}

wnids_pc=[map_parent_child(x) for x in wnids_pc if x!='']
wnids_pc2=pd.DataFrame(wnids_pc)
wnids_pc2.to_csv('wnids_parent_child.csv')

In [191]:
#get all children to the wnid we chose.
children=[]
parents=[]
for set_ in wnids_pc:
    p=set_['parent']
    c=set_['child']
    #print(p,c)
    if(p==wnid):
        children.append(c)
    if(c==wnid):
        parents.append(p)
        
print(children)   
print(parents)

[]
['n07705931']


In [192]:
children_glosses=[]
children_words=[]

parent_glosses=[]
parent_words=[]

if children:
    #loop through and get their lists of glosses and words.
    children_df=xml_dataframe[xml_dataframe['wnid'].isin(children)]
    children_glosses=children_df['gloss'].tolist()
    children_words=children_df['words'].tolist()
    
if parents:
    #loop through and get their lists of glosses and words.
    parents_df=xml_dataframe[xml_dataframe['wnid'].isin(parents)]
    parent_glosses=parents_df['gloss'].tolist()
    parent_words=parents_df['words'].tolist()    

print(children_glosses,children_words)
print(parent_glosses,parent_words)

[] []
['edible reproductive body of a seed plant especially one having sweet flesh'] ['edible fruit']
